## Disclaimer

**Environment:** This notebook has been developped for running with:

- Julia v1.11.x
- JuMP.jl v1.23.x
- GLPK.jl v1.2.1
- HiGHS v1.12.1

**Author:** Xavier Gandibleux, Nantes (France)


----

## Exercise 1

Given the 01 Unidimensional Knapsack Problem (01UKP) formulated by 
$$z=\max\big\{px \mid wx \le c, \ x\in\{0,1\}^n\big\}$$

and the numerical instance corresponding to 
$$n=5$$
$$p=(5, 3, 2, 7, 4)$$
$$w=(2, 8, 4, 2, 5)$$
$$c=10$$

answer to the following questions:

1. implement an explicit formulation of the 01UKP with JuMP and display the model.
2. compute the optimal solution using GLPK as MIP solver and display the solution.
3. compute the linear relaxation using GLPK as LP solver and display the solution.
4. extract the data from the numerical instance and initialize vectors `p` and `w`, and the scalar `c` with these data.
5. given `p`, `w`, and `c`, implement an implicit formulation of the 01UKP with JuMP, compute and display the optimal solution.
6. write a function `setup01UKP` which return a JuMP model corresponding to a 01UKP for a given instance described by `p`, `w`, and `c`.
7. write a function `displayRes01UKP` which display for of a 01UKP the optimal solution $x^*$ and $z^*=f(x^*)$ on screen.
8. using functions `setup01UKP` and `displayRes01UKP`, compute and display (if it is available) the optimal solution using GLPK as MIP solver.
9. same question as before but using HiGHS as MIP solver.
10. same question as before but turn off the printing output from the solver.
11. save the 01UKP model with their data on a file named `my01UKP.lp`
12. for the function `setup01UKP`, add the type to each parameters.
13. for the function `displayRes01UKP`, add the type to the parameter.
14. write a structure `kpData`gathering together the data of a 01UKP.
15. create an instance `kpData` named `kpInstance` with the data from the numerical instance.
16. write a function `displayDat01UKP` which display for a given instance `kpData` of 01UKP the data on screen.
17. revise the function `setup01UKP` using as parameter a variable of type `kpData`
18. using the function `displayDat01UKP`, display the data of a 01UKP stored in a variable of type `kpData`.
19. write a structure `kpSol`gathering together the solution of a 01UKP.
20. store in the variable `kpLP` and `kpIP` of type `kpSol` respectively the LP and IP optimal solution.
21. using these codes, write a main program named `mySolverKP`devoted to solve the 01UKP.

## Answers:

----
### 1. implement and display an explicit formulation of the 01UKP with JuMP.
*Notions: definition of an explicit model with JuMP*

<ins>indications:</ins> an explicit formulation is a description in extension of a mathematical model (composed of variable(s), function(s), constraint(s)), and data. 
<br>
<ins>Hints:</ins> the formulation to implement in JuMP is the following:
$$ \begin{aligned}
\max\quad & 5 x_1 + 3 x_2 + 2 x_3 + 7 x_4 + 4 x_5\\
\text{Subject to} \quad & 2 x_1 + 8 x_2 + 4 x_3 + 2 x_4 + 5 x_5 \leq 10\\
 & x_1, x_2, x_3, x_4, x_5 \in \{0, 1\}\\
\end{aligned} $$

In [91]:
using JuMP, GLPK

kp = Model(GLPK.Optimizer)
@variable(kp, x1, Bin)
@variable(kp, x2, Bin)
@variable(kp, x3, Bin)
@variable(kp, x4, Bin)
@variable(kp, x5, Bin)
@objective(kp, Max, 5x1 + 3x2 + 2x3 + 7x4 + 4x5)
@constraint(kp, 2x1 + 8x2 + 4x3 + 2x4 + 5x5 ≤ 10)

print(kp)

----
### 2. Compute and display the optimal solution of the 01UKP solved using GLPK as MIP solver. 
*Notions: optimizing a JuMP model, getting and displaying the results*

In [37]:
optimize!(kp)

println("zOpt: ", objective_value(kp))
println("xOpt: ", value(x1),"|1  ", value(x2),"|2  ", value(x3),"|3  ", value(x4),"|4  ", value(x5),"|5")

zOpt: 16.0
xOpt: 1.0|1  0.0|2  0.0|3  1.0|4  1.0|5


<br>
<ins> Remark:</ins> in general, a MIP solver manages all decision variables as floating type variables. The values returned may be an approximation of integer values. Refine the output consequently. 

<ins> Hints:</ins> see functions `convert` and `round`.

In [38]:
convert(Int,value(x1))

1

In [41]:
convert(Int,0.999)

LoadError: InexactError: Int64(0.999)

In [44]:
round(0.999,digits=0)

1.0

In [43]:
convert(Int, round(0.999,digits=0))

1

----
### 3. compute the linear relaxation using GLPK as LP solver and display the solution.
*Notions: relaxing all discrete variables of a JuMP model*

<ins>Hints:</ins> see `relax_integrality`

In [ ]:
relax = relax_integrality(kp) # all variables are continuous
print(kp)

optimize!(kp)
println("zOpt: ", objective_value(kp))
println("xOpt:  1| ", value(x1),"  2| ", value(x2),"  3| ", value(x3),"  4| ", value(x4),"  5| ", value(x5))

relax() # restore the definition of variables

----
### 4. extract the data from the numerical instance and initialize vectors `p` and `w`, and the scalar `c` with these data.
*Notions: handling vectors in Julia*

In [3]:
p = [ 5, 3, 2, 7, 4 ]
w = [ 2, 8, 4, 2, 5 ]
c = 10

10

----
### 5. implement an implicit formulation of the 01UKP with JuMP, compute and display the optimal solution.
*Notions: definition of an implicit model with JuMP, resolution, getting the results; "for" instruction in Julia*

<ins>indications:</ins> an implicit formulation is a compact description of a mathematical model (composed of variable(s), function(s), constraint(s)), without data (they are externalized and represented in the formulation by parameters).
<br>
<ins>Hints:</ins> the formulation to implement in JuMP is the following:
$$ \begin{aligned}
\max\quad & \sum_{j=1}^{n} p_{j} x_{j}\\
\text{Subject to} \quad & \sum_{j=1}^{n} w_{j} x_{j} \leq c\\
 & x_{j} \in \{0, 1\} \qquad  j=1,\dots ,n\\
\end{aligned} $$

In [6]:
using Printf

kp = Model(GLPK.Optimizer)
n = length(p)
@variable(kp, x[1:n], Bin)
@objective(kp, Max, sum(p[j]*x[j] for j=1:n))
@constraint(kp, sum(w[j]*x[j] for j=1:n) ≤ c)

optimize!(kp)

println("zOpt: ", objective_value(kp))
print("xOpt: ")
for i = 1:n
    @printf("%1.0f|%1d  ",value(x[i]),i)
end

zOpt: 16.0
xOpt: 1|1  0|2  0|3  1|4  1|5  

----
### 6. write a function `setup01UKP` which return a JuMP model corresponding to a 01UKP for a given instance described by `p`, `w`, and `c`, without specifying the solver to use.
*Notions: handling functions in Julia*

In [7]:
function setup01UKP( p, w, c )
    
    kp = Model( )
    n = length(p)
    @variable(kp, x[1:n], Bin)
    @objective(kp, Max, sum(p[j]*x[j] for j=1:n))
    @constraint(kp, sum(w[j]*x[j] for j=1:n) ≤ c)
    
    return kp
end

setup01UKP (generic function with 1 method)

----
### 7. write a function `displayRes01UKP` which display for of a 01UKP the optimal solution $x^*$ and $z^*=f(x^*)$ on screen.
*Notions: refering the variables of a JuMP model* 

In [10]:
function displayRes01UKP(kp)
    
    println("zOpt: ", objective_value( kp ))
    print("xOpt: ")
    for i = 1:n
        @printf("%1.0f|%1d  ",value( kp[:x][i]),i)
    end
    
    return nothing
end

displayRes01UKP (generic function with 1 method)

----
### 8. using functions `setup01UKP` and `displayRes01UKP`, compute and display the optimal solution (if it is available) using GLPK as MIP solver.
*Notions: using user-defined functions; "if" instruction in Julia*

In [11]:
kp = setup01UKP( p, w, c )
print(kp)

set_optimizer(kp, GLPK.Optimizer)
optimize!(kp)

if is_solved_and_feasible(kp)
    displayRes01UKP(kp)
else
    println("No solution available")
end

zOpt: 16.0
xOpt: 1|1  0|2  0|3  1|4  1|5  

----
### 9. same question as before but using HiGHS as MIP solver.
*Notions: changing the MIP solver to use by JuMP*

In [12]:
using HiGHS

set_optimizer(kp, HiGHS.Optimizer)
optimize!(kp)

if is_solved_and_feasible(kp)
    displayRes01UKP(kp)
else
    println("No solution available")
end

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [2e+00, 8e+00]
  Cost   [2e+00, 7e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+01, 1e+01]
Presolving model
1 rows, 5 cols, 5 nonzeros  0s
1 rows, 4 cols, 4 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   1 rows
   4 cols (4 binary, 0 integer, 0 implied int., 0 continuous)
   4 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

 z       0       0         0   0.

----
### 10. same question as before but turn off the printing output from the solver.
*Notions: general options given to the MIP solver*

In [13]:
set_silent(kp)
optimize!(kp)

if is_solved_and_feasible(kp)
    displayRes01UKP(kp)
else
    println("No solution available")
end

zOpt: 16.0
xOpt: 1|1  0|2  -0|3  1|4  1|5  

----
### 11. save the 01UKP model with their data on a file named `my01UKP.lp`
*Notions: saving a JuMP model on a file*

In [14]:
write_to_file(kp,"my01UKP.lp")

----
### 12. for the function `setup01UKP`, add the type to each parameters.
*Notions: types in Julia; typing a variable in Julia*

<ins> Hints:</ins> see the function `typeof`

In [15]:
typeof(p)

Vector{Int64} (alias for Array{Int64, 1})

In [16]:
typeof(w)

Vector{Int64} (alias for Array{Int64, 1})

In [17]:
typeof(c)

Int64

In [18]:
function setup01UKP( p::Vector{Int64}, w::Vector{Int64}, c::Int64 )
    
    kp = Model( )
    n = length(p)
    @variable(kp, x[1:n], Bin)
    @objective(kp, Max, sum(p[j]*x[j] for j=1:n))
    @constraint(kp, sum(w[j]*x[j] for j=1:n) ≤ c)
    
    return kp
end

setup01UKP (generic function with 2 methods)

----
### 13. for the function `displayRes01UKP`, add the type to the parameter.
*Notions: type of a JuMP model*

In [20]:
typeof(kp)

Model (alias for GenericModel{Float64})

In [19]:
function displayRes01UKP(kp::Model)
    
    println("zOpt: ", objective_value( kp ))
    print("xOpt: ")
    for i = 1:n
        @printf(" %1d|%1.0f ",i, value( kp[:x][i]) )
    end
    
    return nothing
end

displayRes01UKP (generic function with 2 methods)

----
### 14. write a structure `kpData`gathering together the data of a 01UKP.
*Notions: structures (objects) in Julia*

In [21]:
mutable struct kpData
    p::Vector{Int64}
    w::Vector{Int64}
    c::Int64
end

----
### 15. create an instance `kpData` named `kpInstance` with the data from the numerical instance.
*Notions: variable instanciated from a structure (object)*

In [22]:
kpInstance = kpData([5, 3, 2, 7, 4], [2, 8, 4, 2, 5], 10)

kpData([5, 3, 2, 7, 4], [2, 8, 4, 2, 5], 10)

----
### 16. write a function `displayDat01UKP` which display for of a 01UKP the data on screen.
*Notions: accessing to a member of a structure*

In [74]:
function displayDat01UKP(instance::kpData)

    n = length(instance.p)
    
    print("i: ")
    for i = 1:n
        @printf("%3.0f ", i)
    end
    print("\np: ")
    for i = 1:n
        @printf("%3.0f ", instance.p[i])
    end
    print("\nw: ")
    for i = 1:n
        @printf("%3.0f ", instance.w[i])
    end    
    println("  |  c:  ",instance.c)
    
    return nothing
end

displayDat01UKP (generic function with 1 method)

----
### 17. revise the function `setup01UKP` using as parameter a variable of type `kpData`
*Notions: function and methods; multiple dispatch*

In [35]:
function setup01UKP(instance::kpData)
    
    kp = Model( )
    n = length(instance.p)
    @variable(kp, x[1:n], Bin)
    @objective(kp, Max, sum(instance.p[j]*x[j] for j=1:n))
    @constraint(kp, sum(instance.w[j]*x[j] for j=1:n) ≤ instance.c)
    
    return kp
end

setup01UKP (generic function with 3 methods)

----
### 18. using the function `displayDat01UKP`, display the data of a 01UKP stored in a variable of type `kpData`.

In [75]:
displayDat01UKP(kpInstance)

i:   1   2   3   4   5 
p:   5   3   2   7   4 
w:   2   8   4   2   5   |  c:  10


----
### 19. write a structure `kpSol`gathering together the solution of a 01UKP.
*Notions: hierarchy of types; parametric structure*

<ins> Hints:</ins> see functions `supertype` and `subtypes`

In [88]:
supertype(Real)

Number

In [89]:
subtypes(Real)

5-element Vector{Any}:
 AbstractFloat
 AbstractIrrational
 ForwardDiff.Dual
 Integer
 Rational

In [29]:
mutable struct kpSol{T<:Real}
    z::T
    x::Vector{T}
end

----
### 20. store in the variable `kpLP` and `kpIP` of type `kpSol` respectively the LP and IP optimal solution.
*Notions: casting to a given type*

In [30]:
convert(Int,5.0)

5

In [31]:
convert.(Int, [5.0,3.0])

2-element Vector{Int64}:
 5
 3

In [64]:
kp = setup01UKP( p, w, c )
n = length(kpInstance.p)
set_optimizer(kp, GLPK.Optimizer)

In [60]:
# 0≤x≤1
relax = relax_integrality(kp) 
optimize!(kp)

kpOpt = kpSol(0.0,zeros(n))
kpOpt.z = objective_value(kp)
kpOpt.x = value.(kp[:x])

relax()

@show kpOpt

kpOpt = kpSol{Float64}(16.5, [1.0, 0.0, 0.25, 1.0, 1.0])


kpSol{Float64}(16.5, [1.0, 0.0, 0.25, 1.0, 1.0])

In [65]:
# x ∈ {0,1}
optimize!(kp)

kpOpt = kpSol(0,zeros(Int,n))
kpOpt.z = convert(Int, objective_value(kp))
kpOpt.x = convert.(Int,value.(kp[:x]))

@show kpOpt

kpOpt = kpSol{Int64}(16, [1, 0, 0, 1, 1])


kpSol{Int64}(16, [1, 0, 0, 1, 1])

In [66]:
function getSolution(kp::Model)
    
    n = length(all_variables(kp))

    if is_binary(kp[:x][1])        
        kpOpt = kpSol(0, zeros(Int,n))
        kpOpt.z = convert(Int, objective_value(kp))
        kpOpt.x = convert.(Int,value.(kp[:x]))
        return kpOpt
    else
        kpOpt = kpSol(0.0, zeros(n))
        kpOpt.z = objective_value(kp)
        kpOpt.x = value.(kp[:x])
        return kpOpt
    end
end        

getSolution (generic function with 2 methods)

In [85]:
function displayRes01UKP(solution::kpSol)
    
    println("zOpt: ", solution.z)
    print("xOpt: ")
    for i = 1:length(solution.x)
        @printf("%1.0f|%1d  ",solution.x[i],i)
    end
    
    return nothing
end

displayRes01UKP (generic function with 3 methods)

----
### 21. using this learning, implement the required codes to solve instances of the Linear Assignment Problem (LAP).


In [112]:
using Printf, JuMP, GLPK
function mySolverLAP()

    c  = [ 3 9 0 0 6 ; 16 0 6 12 19 ; 2 7 11 15 8 ; 4 11 7 16 3 ; 2 5 1 9 0 ]
    n = size(c,1)

    lap = Model( )
    @variable( lap, x[1:n,1:n] , Bin )
    @objective( lap, Min, sum( c[i,j]*x[i,j] for i=1:n,j=1:n ))
    @constraint( lap, cols[i=1:n], sum(x[i,j] for j=1:n) == 1 )
    @constraint( lap, rows[j=1:n], sum(x[i,j] for i=1:n) == 1 )
    
    set_optimizer(lap, GLPK.Optimizer)
    optimize!(lap)
  
    if is_solved_and_feasible(lap)
        println("zOpt: ", objective_value( lap ))
        print("xOpt: ")
        for i = 1:n, j =1:n
            if value(lap[:x][i,j]) > 0.5 
                @printf("x[%1d,%1d]=1  ", i, j)
            end
        end
    end

    return nothing
end
mySolverLAP()

zOpt: 6.0
xOpt: x[1,4]=1  x[2,2]=1  x[3,1]=1  x[4,5]=1  x[5,3]=1  